In [1]:
include("../LiPoSID.jl")
include("CtrlSID.jl")

using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using HDF5
using DynamicPolynomials

using Dates
using Statistics

┌ Warning: Package LinearSolve does not have KrylovKit in its dependencies:
│ - If you have LinearSolve checked out for development and have
│   added KrylovKit as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LinearSolve
└ Loading KrylovKit into LinearSolve from project dependency, future warnings for LinearSolve are suppressed.


Consider Kossakowski constraints [1] for two-level system (Theorem 3.1 on page 824).

### References

1. **Vittorio Gorini, Andrzej Kossakowski, and E. C. G. Sudarshan (1976)** "Completely positive dynamical semigroups of Nlevel systems" *J. Math. Phys.* 17, 821 (1976) DOI: [10.1063/1.522979](https://doi.org/10.1063/1.522979)

In [2]:
σˣ = [ 0 1 
       1 0 ]

σʸ = [ 0.   im*1
      -im*1 0    ]

σᶻ = [ 1.  0
       0  -1 ] 

fᴷ₁ = σˣ/2; fᴷ₂ = σʸ/2; fᴷ₃ = σᶻ/2

@assert tr(σˣ/2*σʸ/2) == tr(σˣ/2*σᶻ/2) ==  tr(σʸ/2*σᶻ/2) ≈ 0
@assert tr(σˣ/2*σˣ/2) == tr(σʸ/2*σʸ/2) == tr(σᶻ/2*σᶻ/2) ≈ 1/2

fᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

@polyvar γ[1:3]
@polyvar a[1:3]

Cˢʸᵐᵇ = [ -γ[1]+γ[2]+γ[3]   -im*a[3]          im*a[2]+0.0
           im*a[3]           γ[1]-γ[2]+γ[3]  -im*a[1] 
          -im*a[2]           im*a[1]          γ[1]+γ[2]-γ[3]] 

κ₁ = -γ[1]+γ[2]+γ[3]
κ₂ =  γ[1]-γ[2]+γ[3]
κ₃ =  γ[1]+γ[2]-γ[3]

constr1 = κ₁ + κ₂ + κ₃  
constr2 = κ₁*κ₂ + κ₃*κ₁ + κ₂*κ₃ - a[1]^2 - a[2]^2 - a[3]^2
constr3 = κ₁*κ₂*κ₃ - κ₁*a[1]^2 - κ₂*a[2]^2 - κ₃*a[3]^2 

kossak_constrs = [κ₁, κ₂, κ₃, γ[1], γ[2], γ[3], constr1, constr2, constr3]

@polyvar ϵ h_Re h_Im # h₁ h₂ h₃

Hˢʸᵐᵇ = [ ϵ               h_Re+im*h_Im
           h_Re-im*h_Im   -ϵ            ] / 2

Hᴷˢʸᵐᵇ = h_Re * fᴷ₁ + h_Im * fᴷ₂  + ϵ * fᴷ₃ 

@assert tr(Hᴷˢʸᵐᵇ) == 0
@assert Hᴷˢʸᵐᵇ == Hˢʸᵐᵇ

### Auxilary functions

Fidelity for qubit can be written:

$F(\rho, \sigma) = \operatorname{tr}(\rho \sigma) + 2\sqrt{\det(\rho) \det(\sigma)}$

If we need fidelity more then certain threshold $F$ we have constraint:

$4\det(\rho) \det(\sigma) - F^2 + 2 F \operatorname{tr}(\rho \sigma) - \operatorname{tr}(\rho \sigma)^2 > 0$

It is more convenient to use infidelity therhold $\Delta = 1-F \ll 1$

In [3]:
function infidelity_constr(ρ, σ, Δ) #for two-level system
    F = 1-Δ
    return 4*(det(ρ)*det(σ)) - F^2 + 2*F*tr(ρ*σ) - tr(ρ*σ)^2
end



infidelity_constr (generic function with 1 method)

$4\det(\rho) \det(\sigma) + 2\operatorname{tr}(\rho \sigma)- \operatorname{tr}(\rho \sigma)^2 - 1 > -2\Delta+\Delta^2 +\Delta\operatorname{tr}(\rho \sigma)$

$4\det(\rho) \det(\sigma) + 2\operatorname{tr}(\rho \sigma)- \operatorname{tr}(\rho \sigma)^2 - 1 > \Delta^2 +\Delta(\operatorname{tr}(\rho \sigma) -2)$

$1 - 4\det(\rho) \det(\sigma) - 2\operatorname{tr}(\rho \sigma) + \operatorname{tr}(\rho \sigma)^2 < \Delta(2-\operatorname{tr}(\rho \sigma)) - \Delta^2$



$\frac{1 - 4\det(\rho) \det(\sigma) - 2\operatorname{tr}(\rho \sigma) + \operatorname{tr}(\rho \sigma)^2}
{2-\operatorname{tr}(\rho \sigma)} < \Delta - \frac{\Delta^2}{2-\operatorname{tr}(\rho \sigma)}$

as $\operatorname{tr}(\rho \sigma) \leq 1$ then $2-\operatorname{tr}(\rho \sigma) \in [1,2]$ for $\Delta < 1$:

$\Delta - \frac{\Delta^2}{2-\operatorname{tr}(\rho \sigma)} < \Delta - \Delta^2 < 1$


As $1/x = 1 - (x-1) + (x-1)^2 - (x-1)^3...$  :

$\frac{1 - 4\det(\rho) \det(\sigma) - 2\operatorname{tr}(\rho \sigma) + \operatorname{tr}(\rho \sigma)^2}
{2-\operatorname{tr}(\rho \sigma)} = \left(1 - 4\det(\rho) \det(\sigma) - 2\operatorname{tr}(\rho \sigma) + \operatorname{tr}(\rho \sigma)^2\right)(1-(1-\operatorname{tr}(\rho \sigma))+(1-\operatorname{tr}(\rho \sigma))^2...)$

We can write fown approximate infidelity loss function:

$L(\rho,\sigma)= \left(1 - 4\det(\rho) \det(\sigma) - 2\operatorname{tr}(\rho \sigma) + \operatorname{tr}(\rho \sigma)^2\right)(1-(1-\operatorname{tr}(\rho \sigma)))$

$L(\rho,\sigma)= 1 - 4\det(\rho) \det(\sigma) - 2\operatorname{tr}(\rho \sigma) + \operatorname{tr}(\rho \sigma)^2$

In [4]:
function infidelity_norm(ρ, σ)
    (1-4*(det(ρ)*det(σ))-2*tr(ρ * σ) + tr(ρ * σ)^2)#*(1 -(1-tr(ρ * σ)))#+(1-tr(ρ * σ))^2)#-(1-tr(ρ * σ))^3)
end

infidelity_norm (generic function with 1 method)

Right hand side of the GKSL equation in GKS form:

$\frac{d\rho}{dt} = -i[H,\rho] + \frac{1}{2} \sum_{i,j}^3 C_{ij} \{[f_i, \rho f_j] + [f_i\rho, f_j]\}, \quad \rho \in M(2)$

where $C \succeq 0$ - semidefinite matrix

In [5]:
function Dc(ρ, t)
    U = (Hˢʸᵐᵇ*ρ - ρ*Hˢʸᵐᵇ)/im 
    D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in fᴼᴺᴮ, fⱼ in fᴼᴺᴮ])/2
    return U + D
end 

Dc (generic function with 1 method)

In [6]:
function reassemble_poly(p)
    if isempty(monomials(p))
        p = 0. 
    else
        poly = sum(real(p) * mon for (coef, mon) in zip(coefficients(p), monomials(p)))
    end
    return poly
end

reassemble_poly (generic function with 1 method)

In [7]:
function kossak_infid_obj(ρ, t)

    obj = 0
    for i in 3:length(ρ)
        ρ1 = ρ[i]
        ρ2 = ρ[i-2] + (t[i]-t[i-1])*(Dc(ρ[i],t[i]) + 4*Dc(ρ[i-1],t[i-1]) + Dc(ρ[i-2],t[i-2])) / 3
        obj += infidelity_norm(ρ1,ρ2)^2
    end

    obj = reassemble_poly(obj)

    return obj

end

kossak_infid_obj (generic function with 1 method)

Read Kurt's spin-boson system data 

In [8]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

function read_GEXY_timeevolution(file_name, γ)

    tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γ)
    tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γ)
    tˣ, ρˣ = read_timeevolution(file_name, "B3", γ)
    tʸ, ρʸ = read_timeevolution(file_name, "B4", γ)

    ρᵍᵉˣʸ = ρᵍ, ρᵉ, ρˣ, ρʸ 
    tᵍᵉˣʸ = tᵍ, tᵉ, tˣ, tʸ

    return tᵍᵉˣʸ , ρᵍᵉˣʸ 

end

read_GEXY_timeevolution (generic function with 1 method)

Assemble objective and constraints using Kossakowski and fidelity constraints

In [9]:
function kossak_GEXY_infid_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = kossak_infid_obj(ρᵍ, tᵍ)
    polyE = kossak_infid_obj(ρᵉ, tᵉ)
    polyX = kossak_infid_obj(ρˣ, tˣ)
    polyY = kossak_infid_obj(ρʸ, tʸ)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end

kossak_GEXY_infid_obj (generic function with 1 method)

Load data

In [10]:
file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"
#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

γᵢ = "0.25133"

tᵍᵉˣʸ , ρᵍᵉˣʸ  = read_GEXY_timeevolution(file_name, γᵢ);

Assemble objective and constraints

In [11]:
polyGEXY = kossak_GEXY_infid_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ)

2.3113668140598527e-7γ₁⁸ + 3.363641225558502e-7γ₁⁷γ₂ + 3.808291770720158e-7γ₁⁷γ₃ + 2.1360726765951632e-7γ₁⁷a₁ + 2.2633810895509177e-7γ₁⁷a₂ + 5.781865071775712e-7γ₁⁷a₃ + 8.120559279287325e-8γ₁⁷ϵ + 2.310388670230876e-7γ₁⁷h_Re + 2.3119287402402947e-7γ₁⁷h_Im + 4.468031968889212e-7γ₁⁶γ₂² + 5.581114423321801e-7γ₁⁶γ₂γ₃ + 3.090143831389065e-7γ₁⁶γ₂a₁ + 3.2492522523204486e-7γ₁⁶γ₂a₂ + 8.547334975133624e-7γ₁⁶γ₂a₃ + 1.2324077635996112e-7γ₁⁶γ₂ϵ + 3.3621283599383395e-7γ₁⁶γ₂h_Re + 3.364501807737321e-7γ₁⁶γ₂h_Im + 6.572735394767151e-7γ₁⁶γ₃² + 3.632756783021442e-7γ₁⁶γ₃a₁ + 3.7607035903390137e-7γ₁⁶γ₃a₂ + 1.5880646393113242e-6γ₁⁶γ₃a₃ + 1.5588688666974364e-7γ₁⁶γ₃ϵ + 3.8072973192538114e-7γ₁⁶γ₃h_Re + 3.8089802277480417e-7γ₁⁶γ₃h_Im + 2.694136926991671e-6γ₁⁶a₁² - 9.353039966779529e-8γ₁⁶a₁a₂ + 5.588099199079545e-7γ₁⁶a₁a₃ + 7.456183309136348e-8γ₁⁶a₁ϵ + 3.8857006871179305e-7γ₁⁶a₁h_Re + 8.01793116365337e-7γ₁⁶a₁h_Im + 7.852343051088752e-7γ₁⁶a₂² + 5.735955274225942e-7γ₁⁶a₂a₃ + 7.958507026906995e-8γ₁⁶a₂ϵ + 1.737292993

In [12]:
@show minimum(abs.(coefficients(polyGEXY)))
@show maximum(abs.(coefficients(polyGEXY)))
maxdegree(polyGEXY)

minimum(abs.(coefficients(polyGEXY))) = 8.298826851285467e-11
maximum(abs.(coefficients(polyGEXY))) = 0.022182963683321552


8

Optimization

In [13]:
using TSSOS

In [15]:
vars = variables(polyGEXY)
opt,sol,data = cs_tssos_first([polyGEXY, kossak_constrs...], vars, 4, solution=true)#, mosek_setting=settings)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[9]
[1]
-----------------------------------------------------------------------------
Obtained the variable cliques in 0.017184801 seconds. The maximal size of cliques is 9.
Starting to compute the block structure...
Obtained the block structure in 0.379532966 seconds.
The maximal size of blocks is 715.
Assembling the SDP...
There are 24310 affine constraints.
SDP assembling time: 5.663284469 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 24310           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 1      

(0.009252144936957417, [9.764059845912195e-7, 9.791090441844195e-7, 9.865186565778293e-7, -5.5613562747186155e-5, -5.6421746261853976e-5, -6.064069725960585e-5, -0.2849783737416659, -0.3602491565825182, -0.36025850771180523], TSSOS.mcpop_data(9, 0, 9, 0, Vector{Vector{UInt16}}[[[], [0x0001], [0x0001, 0x0001], [0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0001, 0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0001, 0x0001, 0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0001, 0x0001, 0x0001, 0x0001, 0x0002]  …  [0x0008, 0x0009, 0x0009, 0x0009, 0x0009, 0x0009, 0x0009], [0x0008, 0x0009, 0x0009, 0x0009, 0x0009, 0x0009, 0x0009, 0x0009], [0x0009], [0x0009, 0x0009], [0x0009, 0x0009, 0x0009], [0x0009, 0x0009, 0x0009, 0x0009], [0x0009, 0x0009, 0x0009, 0x0009, 0x0009], [0x0009, 0x0009, 0x0009, 0x0009, 0x0009, 0x0009], [0x0009, 0x0009, 0x0009, 0x0009, 0x0009,

In [16]:
solution = variables(polyGEXY) => sol

Cˢⁱᵈ = subs(Cˢʸᵐᵇ, solution)
Hˢⁱᵈ = subs(Hˢʸᵐᵇ, solution)

Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (9.89222e-7+0.0im)   (-0.0+6.06407e-5im)
 (-0.0-6.06407e-5im)  (9.83816e-7+0.0im)

In [ ]:
Hˢⁱᵈ

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (12.5724+0.0im)              (0.000167566-0.000177018im)
 (0.000167566+0.000177018im)  (-12.5724+0.0im)

In [17]:
Hˢⁱᵈ = convert.(ComplexF64,Hˢⁱᵈ)
Cˢⁱᵈ = convert.(ComplexF64,Cˢⁱᵈ)

3×3 Matrix{ComplexF64}:
 9.89222e-7+0.0im                0.0+6.06407e-5im         0.0-5.64217e-5im
        0.0-6.06407e-5im  9.83816e-7+0.0im                0.0+5.56136e-5im
        0.0+5.64217e-5im         0.0-5.56136e-5im  9.68996e-7+0.0im

In [18]:
function get_lindblad_operators(C::Matrix{ComplexF64}, basis_ops::Vector{Matrix{ComplexF64}})
    # Check that C is a square matrix and basis_ops has the same dimension
    n = size(C, 1)
    if size(C, 2) != n || length(basis_ops) != n
        throw(ArgumentError("Dimensions of C and basis_ops do not match"))
    end

    # Perform eigenvalue decomposition of C
    eigvals, eigvecs = eigen(C)

    # Construct the Lindblad operators
    lindblad_ops = []
    for i in 1:n
        if eigvals[i] > 1e-10  # Filter out negligible eigenvalues to ensure numerical stability
            lindblad_op = zeros(ComplexF64, size(basis_ops[1]))
            for j in 1:n
                lindblad_op .+= sqrt(eigvals[i]) * eigvecs[j, i] * basis_ops[j]
            end
            push!(lindblad_ops, lindblad_op)
        end
    end

    return lindblad_ops
end

effective_Lindblad = get_lindblad_operators(Cˢⁱᵈ, fᴼᴺᴮ)

2-element Vector{Any}:
 ComplexF64[-0.0003008584479277803 + 0.0im -0.0002758753168815986 - 0.0002799093282891042im; -0.000275959048053374 + 0.0002799443652868874im 0.0003008584479277803 + 0.0im]
 ComplexF64[0.0028177081157141105 + 0.0im -0.004005484015040272 - 0.004063829191358789im; 0.00097686551829623 - 0.0009911984173863765im -0.0028177081157141105 + 0.0im]

In [19]:
ρᵍ₀ = [ 1 0.
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states;

In [20]:
FminStates = []
FmedianStates = []
FmeanStates = []

for state in test_states # loop over initial states
    
    print(state*" ")

    start_time = time()

    tₛ, ρₛ = read_timeevolution(file_name, state, γᵢ)
    ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
    #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
    ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
    tᵗˢᵗ = convert.(Float64, tₛ)

    #Simulated LME 
    #tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵈ), [Jˢⁱᵐ])
    #bˢⁱᵐ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵐ])

    ρₒ = DenseOperator(basis,ρₛ[1])
    dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
    tᵉⁿᵈ = tᵗˢᵗ[end]

    #print("effective_Lindblad_ops for Kossakowski")

    
    effective_Lindblad_ops = [DenseOperator(basis,j) for j in effective_Lindblad]

    #print("Simulating Kossakowski")

    tout, ρ_t_kossak = timeevolution.master(tᵗˢᵗ, ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
    ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

    #print("Calculating Fidelity")

    #F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
    F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)
    
    FminState = minimum(F)
    FmedianState = median(F)
    FmeanState = mean(F)
    
    push!(FminStates, FminState)
    push!(FmedianStates, FmedianState)
    push!(FmeanStates, FmeanState)

end

# Calculate the mean
F_mean_value = mean(FmeanStates)

# Calculate the median
F_median_value = median(FmedianStates)

# Calculate the min
F_min_value = minimum(FminStates)

println()
println("Mimimal fidelity for "*γᵢ*": ", F_min_value)
println("Median fidelity for "*γᵢ*": ", F_median_value)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.25133: 0.1119967910406339
Median fidelity for 0.25133: 0.722372396827416
